<a href="https://colab.research.google.com/github/ag4267research1/Hybrid-Quantum-PDE-constrained-Optimization/blob/main/PDE_constrained_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

We control the source term in

$$
-u^{\prime \prime}(x)=z(x), \quad x \in(0,1), \quad u(0)=u(1)=0,
$$

where
- $u(x)=$ state,
- $z(x)=$ control (what we can choose).

We want $u$ to track a desired state $u_d(x)$ and we penalize large controls:

$$
J(u, z)=\frac{1}{2} \int_0^1\left(u(x)-u_d(x)\right)^2 d x+\frac{\alpha}{2} \int_0^1 z(x)^2 d x, \quad \alpha>0
$$


Goal (single step):
For a given control $z$, compute
1. state $u$ (solve PDE),
2. adjoint $p$,
3. gradient w.r.t. control $g=\frac{d J}{d z}$.

We will then solve the adjoint equation both classically and with a toy QLSA.

- State equation (from variation in $p$ ):

$$
-u^{\prime \prime}=z, \quad u(0)=u(1)=0 .
$$

- Adjoint equation (from variation in $u$ ):

$$
-p^{\prime \prime}=u-u_d, \quad p(0)=p(1)=0 .
$$

- Gradient (from variation in $z$ ):

$$
\frac{d J}{d z}=\alpha z-p .
$$


Now discretize this in 1D with a tiny grid so we can do quantum stuff.

Grid:
- Points: $x_0=0, x_1=1 / 3, x_2=2 / 3, x_3=1$
- Interior points: $x_1, x_2$
- Grid spacing $h=1 / 3$
- Unknowns:

$$
\begin{aligned}
& u_1 \approx u\left(x_1\right), u_2 \approx u\left(x_2\right) \\
& z_1 \approx z\left(x_1\right), z_2 \approx z\left(x_2\right) \\
& p_1 \approx p\left(x_1\right), p_2 \approx p\left(x_2\right)
\end{aligned}
$$


Central difference for the Laplacian:

$$
-u^{\prime \prime}\left(x_i\right) \approx-\frac{u_{i-1}-2 u_i+u_{i+1}}{h^2} .
$$


So the discrete state equation at interior nodes is:

$$
-\frac{u_{i-1}-2 u_i+u_{i+1}}{h^2}=z_i \quad \Rightarrow \quad-u_{i-1}+2 u_i-u_{i+1}=h^2 z_i .
$$


With boundary values $u_0=u_3=0$, the system becomes:

$$
\begin{aligned}
2 u_1-u_2 & =h^2 z_1, \\
-u_1+2 u_2 & =h^2 z_2 .
\end{aligned}
$$


In matrix form:

$$
A u=h^2 z, \quad A=\left(\begin{array}{cc}
2 & -1 \\
-1 & 2
\end{array}\right), \quad u=\binom{u_1}{u_2}, z=\binom{z_1}{z_2} .
$$


Now discretize the adjoint equation $-p^{\prime \prime}=u-u_d$ in the same way:

$$
A p=-h^2\left(u-u_d\right) .
$$

So the same matrix $A$ appears in both state and adjoint.
Finally, the discrete gradient is

$$
g=\frac{d J}{d z}=\alpha h^2 z-p,
$$

all understood component-wise.

To make everything explicit:
- Take $N=3 \rightarrow \mathrm{~h}=1 / 3$.
- Choose a simple control (just to have numbers), e.g.

$$
z=\binom{1}{0} .
$$

- Choose desired state $u_d(x)=\sin (\pi x)$.

At interior nodes:

$$
u_{d, 1}=\sin (\pi / 3), \quad u_{d, 2}=\sin (2 \pi / 3) .
$$


Now we can:
1. Solve state $A u=h^2 z$.
2. Solve adjoint $A p=-h^2\left(u-u_d\right)$.
3. Compute gradient $g=\alpha h^2 z-p$.

In [ ]:
import numpy as np
import time

# Parameters
h = 1.0/3.0
alpha = 1e-2

# Matrix A (discrete Laplacian with Dirichlet BC)
A = np.array([[2.0, -1.0],
              [-1.0, 2.0]])

# Control z (2 interior nodes)
z = np.array([1.0, 0.0])

# Desired state u_d(x) = sin(pi x)
x1, x2 = 1.0/3.0, 2.0/3.0
u_d = np.array([np.sin(np.pi * x1), np.sin(np.pi * x2)])

# -------------------------------
# 1. State solve: A u = h^2 z
# -------------------------------
t0 = time.time()
u = np.linalg.solve(A, (h**2) * z)
t1 = time.time()

print("State u =", u)
print("State solve time (s) =", t1 - t0)

# -------------------------------
# 2. Adjoint solve: A p = -h^2 (u - u_d)
# -------------------------------
rhs_adj = - (h**2) * (u - u_d)

t0 = time.time()
p = np.linalg.solve(A, rhs_adj)
t1 = time.time()

print("\nAdjoint RHS =", rhs_adj)
print("Adjoint p   =", p)
print("Adjoint solve time (s) =", t1 - t0)

# -------------------------------
# 3. Gradient wrt control:
#    g = alpha h^2 z - p
# -------------------------------
g = alpha * (h**2) * z - p

print("\nGradient dJ/dz =", g)


State u = [0.07407407 0.03703704]
State solve time (s) = 0.0009582042694091797

Adjoint RHS = [0.08799459 0.09210982]
Adjoint p   = [0.08936633 0.09073808]
Adjoint solve time (s) = 0.000118255615234375

Gradient dJ/dz = [-0.08825522 -0.09073808]


 Quantum part: QLSA adjoint solve

Now we want a quantum linear solver (QLSA) to solve the adjoint system

$$
A p=\text { rhs_adj, }
$$

for the same matrix $\boldsymbol{A}$ and RHS.
Because full QSLA is heavy, we implement a minimal 3-qubit circuit that has the correct QSLA structure:
1. encode rhs_adj as a quantum state $|b\rangle$,
2. apply a controlled $e^{i A t}$ (QPE),
3. apply a controlled rotation (encodes $\sim 1 / \lambda$ ),
4. uncompute QPE,
5. inspect the resulting state as an approximation of $|p\rangle$.

In [ ]:
!pip install qiskit qiskit-aer
import numpy as np
from scipy.linalg import expm
from qiskit import QuantumCircuit, transpile
from qiskit.circuit.library import Initialize
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import Aer
from qiskit.quantum_info import DensityMatrix, partial_trace

We want to solve $A p=r h s_{-} a d j$. Quantumly we encode rhs_adj as amplitudes.

In [ ]:
# Use the RHS from the classical adjoint computation above
b_adj = rhs_adj.copy()

# Normalize to get a 1-qubit state |b>
b_norm = np.linalg.norm(b_adj)
if b_norm == 0:
    raise ValueError("Adjoint RHS is zero; trivial solution.")
b_state = b_adj / b_norm

print("Normalized adjoint RHS state |b> =", b_state)

Normalized adjoint RHS state |b> = [0.69076972 0.72307482]


This means

$$
|b\rangle=b_0|0\rangle+b_1|1\rangle, \quad b_0, b_1=\left(b_{-} a d j\right) /\left\|b_{-} a d j\right\| .
$$

### Build U = exp(i A_scaled t)

In [ ]:
# Scale A so that eigenvalues lie in [-1,1] for numerical stability
A_scaled = A / np.linalg.norm(A)
t_param = 1.0

U = expm(1j * A_scaled * t_param)
U_gate = UnitaryGate(U, label="e^{iA}")


### Build the QLSA (HHL-style) circuit for the adjoint system

In [ ]:
def build_QSLA_adjoint_circuit(b_state, U_gate):
    """
    Tiny QLSA circuit for solving A p = rhs_adj:
    - 3 qubits: phase (0), system (1), rotation (2).
    - 1 classical bit: for measuring the rotation ancilla (not used for statevector).
    """
    qc = QuantumCircuit(3, 1)

    q_phase = 0
    q_sys   = 1
    q_rot   = 2

    # 1. Prepare |b> on system qubit
    qc.append(Initialize(b_state), [q_sys])

    # 2. Start QPE: H on phase, then controlled-U
    qc.h(q_phase)
    qc.append(U_gate.control(1), [q_phase, q_sys])

    # 3. Inverse QFT on phase (for 1 qubit: another H)
    qc.h(q_phase)

    # 4. Controlled rotation RY on q_rot, controlled by phase qubit
    theta = np.pi / 3  # toy angle, stands in for 1/lambda dependence
    qc.cry(theta, q_phase, q_rot)

    # 5. Uncompute QPE
    qc.h(q_phase)
    qc.append(U_gate.control(1).inverse(), [q_phase, q_sys])
    qc.h(q_phase)

    # 6. Measure rotation ancilla (success flag)
    qc.measure(q_rot, 0)

    return qc

adjoint_qc = build_QSLA_adjoint_circuit(b_state, U_gate)
print(adjoint_qc.draw('text'))


                  ┌───┐                       ┌───┐           ┌───┐           »
q_0: ─────────────┤ H ├─────────────────■─────┤ H ├─────■─────┤ H ├─────■─────»
     ┌────────────┴───┴────────────┐┌───┴────┐└───┘     │     └───┘┌────┴────┐»
q_1: ┤ Initialize(0.69077,0.72307) ├┤ e^{iA} ├──────────┼──────────┤ Unitary ├»
     └─────────────────────────────┘└────────┘     ┌────┴────┐ ┌─┐ └─────────┘»
q_2: ──────────────────────────────────────────────┤ Ry(π/3) ├─┤M├────────────»
                                                   └─────────┘ └╥┘            »
c: 1/═══════════════════════════════════════════════════════════╩═════════════»
                                                                0             »
«     ┌───┐
«q_0: ┤ H ├
«     └───┘
«q_1: ─────
«          
«q_2: ─────
«          
«c: 1/═════
«          


### Extracting the adjoint solution from the quantum circuit

In [ ]:
backend_sv = Aer.get_backend("aer_simulator")

# Remove measurement, so we can get a statevector
qc_no_meas = adjoint_qc.copy()
qc_no_meas.remove_final_measurements()

# Tell simulator to save the statevector
qc_no_meas.save_statevector()

compiled = transpile(qc_no_meas, backend_sv)
result = backend_sv.run(compiled).result()

state = result.data(0)["statevector"]
print("Full 3-qubit statevector (|phase, sys, rot>):")
print(state)


Full 3-qubit statevector (|phase, sys, rot>):
Statevector([-0.50880455-0.46439648j,  0.00933499-0.01022765j,
             -0.53199832-0.48556592j,  0.01052053-0.01152656j,
             -0.00522802-0.00477172j, -0.03483866+0.03817012j,
             -0.00771696-0.00704343j, -0.03926315+0.04301771j],
            dims=(2, 2, 2))


This is a vector in an 8 -dimensional space corresponding to basis $\left|q_0 q_1 q_2\right\rangle=|p h a s e, s y s, r o t\rangle$.
To see what the system qubit looks like, we can compute its reduced density matrix by tracing out the ancillas.

In [ ]:
rho_full = DensityMatrix(state)
# Trace out phase (qubit 0) and rotation (qubit 2), keep system qubit 1
rho_sys = partial_trace(rho_full, [0, 2])

print("\nReduced state of system qubit (approx adjoint p):")
print(rho_sys)


Reduced state of system qubit (approx adjoint p):
DensityMatrix([[0.4774587 +9.57525250e-19j, 0.49947819-6.18716005e-16j],
               [0.49947819+6.10117731e-16j, 0.5225413 +1.62139540e-17j]],
              dims=(2,))


In [ ]:
evals, evecs = np.linalg.eigh(rho_sys.data)
# Take eigenvector with largest eigenvalue as the "dominant direction"
idx_max = np.argmax(evals)
p_quantum = evecs[:, idx_max]

# Normalize for safety
p_quantum = p_quantum / np.linalg.norm(p_quantum)

print("\nQuantum adjoint direction (system qubit):", p_quantum)



Quantum adjoint direction (system qubit): [-0.69098343+0.00000000e+00j -0.7228706 -8.82993847e-16j]


In [ ]:
p_classical_norm = p / np.linalg.norm(p)
print("Classical adjoint direction:", p_classical_norm)

overlap = np.vdot(p_classical_norm, p_quantum)
print("Overlap <p_classical | p_quantum> =", overlap, " (|.| =", abs(overlap), ")")


Classical adjoint direction: [0.70170084 0.7124717 ]
Overlap <p_classical | p_quantum> = (-0.9998885000575347-6.291081310583783e-16j)  (|.| = 0.9998885000575347 )


In [ ]:
t0 = time.time()
_ = backend_sv.run(compiled).result()
t1 = time.time()
print("Quantum (QSLA) simulation time (s) =", t1 - t0)


Quantum (QSLA) simulation time (s) = 0.019922971725463867


PDE-optimization level
- State PDE: $-u^{\prime \prime}=z, u(0)=u(1)=0$.
- Discretized as $A u=h^2 z$.
- Adjoint PDE: $-p^{\prime \prime}=u-u_d, p(0)=p(1)=0$.
- Discretized as $A p=-h^2\left(u-u_d\right)$.
- Gradient: $g=\alpha h^2 z-p$.

So the adjoint variable $p$ is the solution of a linear system with the same matrix $A$ but a RHS depending on the state and desired state.

Classical side
- We built $A, z$, and $u_d$.
- Solved:
- state: $A u=h^2 z_{\text {, }}$
- adjoint: $A p=-h^2\left(u-u_d\right)$.
- Computed $g=\alpha h^2 z-p$.

Everything is explicit and runs instantly.


Quantum side
- We focused on the adjoint solve:
- RHS rhs_adj from the classical state and desired state.
- Encoded rhs_adj as a quantum state $|b\rangle$.
- Built a HHL-style QLSA circuit:
- phase ancilla (qubit 0) for QPE,
- system qubit (qubit 1) holding the adjoint vector,
- rotation ancilla (qubit 2 ) encoding $1 / \lambda$ behavior.
- Ran QPE + controlled $e^{i A t}+$ controlled rotation + inverse QPE.
- Extracted the reduced state of the system qubit as a quantum representation of the adjoint solution.
- Compared this to the classical adjoint vector.

This is a hybrid adjoint solve: the PDE, grid, and RHS are classical; the linear solve for the adjoint is done by a quantum linear solver, at least in a small toy example.